In [1]:
import numpy as np
import gurobipy as gp

In [2]:
with open('2010_ga_data.grt', 'r') as file:
    lines = file.readlines()
data = []
for line in lines:
    fields = line.split()
    if len(fields) != 0:
        data.append(fields)
data_demand = []
for i in range(len(data)):
    data_demand.append(int(data[i][4]))
with open('2010_ga_data_distance.grd', 'r') as file:
    lines = file.readlines()
data_dist = [int(line.strip()) for line in lines]

In [3]:
# Calculate the total coverage of the stations in the list
def SumCover(selected):
    copy_data_demand = data_demand.copy()
    total_num = len(data_demand)
    cover_demand_final = 0
    for i in selected:
        cover_demand = 0
        for k in range(total_num*i, total_num*(i+1)):
            if (data_dist[k] <= 25):
                cover_demand += copy_data_demand[k%159]
        cover_demand_final += cover_demand
        for k in range(total_num*i, total_num*(i+1)):
            if (data_dist[k] <= 25):
                copy_data_demand[k%159] = 0
    return cover_demand_final

In [4]:
select_num, total_num = 15, len(data_demand)
sites = [i for i in range(total_num)] # candidate locations
nodes = sites # demand nodes
canCover = [[0 for _ in range(159)] for _ in range(159)]
for i in sites:
    for j in range(total_num*i, total_num*(i+1)):
        if (data_dist[j] <= 25):
            canCover[i][j-total_num*i] = 1

In [5]:
no_improve = 0 # To change the alpha value
alpha, Lambda, BUB, BLB = 2, [0]*159, np.inf, -np.inf # initial parameter

In [6]:
while True:
    # MCP model
    MCP = gp.Model("MCP")
    # Vars
    isCover = MCP.addVars(nodes, vtype='B', name=f'isCover') # Z
    isSelect = MCP.addVars(sites, vtype='B', name=f'isSelect') # X
    # Obj.
    MCP.setObjective(
        gp.quicksum(data_demand[i] * isCover[i] for i in nodes) +
        gp.quicksum(Lambda[i] * (gp.quicksum(canCover[i][j]*isSelect[j] for j in sites) - isCover[i]) for i in nodes),
        gp.GRB.MAXIMIZE
    )
    # s.t.
    MCP.addConstr(gp.quicksum(isSelect[j] for j in sites) <= select_num)
    # Solve
    MCP.setParam('OutputFlag', 0)
    MCP.optimize()

    # Record result
    Z, X = [], []
    if MCP.status == gp.GRB.OPTIMAL:
        for v in MCP.getVars():
            if v.varName.startswith("isCover"):
                Z.append(int(v.x))
            else:
                X.append(int(v.x))
    else:
        print("No solution found.")
    covered, selected = [], []
    for i in range(total_num):
        if Z[i] == 1:
            covered.append(i)
        if X[i] == 1:
            selected.append(i)
    
    # Update UB & LB
    LBUB = BUB
    CUB = MCP.objVal
    BUB = min(BUB, CUB)
    BLB = max(BLB, SumCover(selected))

    # Check if the alpha value needs to be changed
    if BUB == LBUB:
        no_improve += 1
        if no_improve == 20:
            no_improve = 0
            alpha /= 2
    
    # Check if termination is needed
    if (BLB != 0) and ((BUB-BLB)/BLB < 0.01):
        break
    
    # Update Lambda value
    numerator = alpha * (CUB-BLB)
    denominator = sum((sum(canCover[i][j] * X[j] for j in sites) - Z[i]) ** 2 for i in nodes)
    step_size = numerator / denominator
    for i in nodes:
        Lambda[i] = max(0, Lambda[i] - step_size*(sum(canCover[i][j] * X[j] for j in sites) - Z[i]))

# Final result
print('Total Cover:', BLB)
print('Select:', selected)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-05
Total Cover: 8600752
Select: [2, 7, 10, 14, 20, 21, 22, 30, 36, 58, 68, 71, 103, 120, 134]
